연습문제

In [1]:
import sys
assert sys.version_info >= (3, 7)
import sklearn
assert sklearn.__version__ >= "1.0.1"

1.

초승달 데이터셋을 이용한 결정트리 모델을 미세 조정한다.

a. 초승달 데이터셋

In [2]:
from sklearn.datasets import make_moons

X_moons, y_moons = make_moons(n_samples=10000, noise=0.4, random_state=42)

In [3]:
X_moons[:5]

array([[ 0.9402914 ,  0.12230559],
       [ 0.12454026, -0.42477546],
       [ 0.26198823,  0.50841438],
       [-0.49523824,  0.07258876],
       [-0.87941281,  0.54937303]])

In [4]:
y_moons[:5]

array([1, 0, 0, 0, 0])

In [5]:
len(X_moons)

10000

b. 훈련셋과 테스트셋으로 쪼개기

In [6]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(X_moons,y_moons,test_size=0.4,random_state=42)

c. 교차검증을 사용하는 그리드 탐색 실행. 힌트: 다양한 max_leaf_nodes 값 활용.

In [27]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

params = {'max_depth': list(range(2, 10)), 'max_leaf_nodes': list(range(2, 30))}
f = GridSearchCV(DecisionTreeClassifier(random_state=42), params)
f.fit(Xtrain, ytrain)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'max_depth': [2, 3, 4, 5, 6, 7, 8, 9],
                         'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                            13, 14, 15, 16, 17, 18, 19, 20, 21,
                                            22, 23, 24, 25, 26, 27, 28, 29]})

최적의 분류기 확인

In [28]:
print(f.score(Xtrain, ytrain), f.score(Xtest, ytest))
f.best_params_

0.8705 0.85925


{'max_depth': 7, 'max_leaf_nodes': 22}

d. 최적 모델을 전체 훈련셋에 대해 훈련을 다시 진행한 다음 테스트셋에 대한 정확도를 확인할 것. 85% ~ 87% 정도의 정확도 나와야 함.

단, GridSearchCV는 자동으로 전체 훈련셋을 대상으로 한 번 더 훈련한다. 이 옵션을 원하지 않는다면 refit=False 옵션을 GridSearchCV 를 선언할 때 지정한다. 따라서 단순히 아래와 같이 바로 테스트셋에 대한 정확도를 확인할 수 있다.

In [31]:
f = GridSearchCV(DecisionTreeClassifier(random_state=42), params)
f.fit(X_moons, y_moons)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'max_depth': [2, 3, 4, 5, 6, 7, 8, 9],
                         'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                            13, 14, 15, 16, 17, 18, 19, 20, 21,
                                            22, 23, 24, 25, 26, 27, 28, 29]})

In [32]:
print(f.score(X_moons, y_moons), f.score(Xtest, ytest))

0.8675 0.8635


2.

랜덤 포레스트(random forest) 모델을 훈련하는 방법을 살펴본다. 랜덤 포레스트는 여러 개의 결정트리로 구성된다.

a. 무작위로 선택된 100개의 초승달 훈련 샘플로 구성된 훈련셋을 1000개 생성한다. 이를 위해 사이킷런의 ShuffleSplit 클래스를 이용한다.

In [34]:
from sklearn.model_selection import ShuffleSplit

rs = ShuffleSplit(n_splits=1000, train_size=100)
models = []
for train_index, _ in rs.split(X_moons):
    X_mini_train = X_moons[train_index]
    y_mini_train = y_moons[train_index]

b. 앞서 찾은 최적의 모델을 각 미니 훈련셋에 대해 추가 훈련한 다음 테스트셋에 대한 정확도의 평균값을 계산한다. 결과는 80% 정도.

clone() 함수: 모델 복제. 동일 모델을 반복해서 사용하지 않기 위해 사용.

In [40]:
from sklearn.base import clone
import numpy as np

scores = []
for train_index, _ in rs.split(X_moons):
    model_clone = clone(f.best_estimator_)
    X_mini_train = X_moons[train_index]
    y_mini_train = y_moons[train_index]
    model_clone.fit(X_mini_train, y_mini_train)
    score = model_clone.score(Xtest, ytest)
    scores.append(score)

mean = np.mean(scores)
print(f"정확도: {mean*100}%")

정확도: 79.96752500000001%


c. 이번에는 그런데 1000개의 모델의 정확도의 평균이 아닌 1000개의 모델이 가장 많이 예측하는 값을 예측값으로 사용해보자.

아래 코드는 테스트셋에 포함된 샘플별로 1000개의 모델이 예측한 값(클래스)의 최빈값(mode)를 계산한다.

d. 이 방식으로 계산된 예측값을 이용하면 정확도가 87% 정도로 상승한다.

y_pred_majority_votes.reshape([-1])는 차원을 줄여 1차원 어레이로 변환한다.

In [41]:
from sklearn.metrics import accuracy_score
from scipy.stats import mode

predictions = []
for train_index, _ in rs.split(X_moons):
    model_clone = clone(f.best_estimator_)
    X_mini_train = X_moons[train_index]
    y_mini_train = y_moons[train_index]
    model_clone.fit(X_mini_train, y_mini_train)
    pred = model_clone.predict(Xtest)
    predictions.append(pred)

mode, _ = mode(predictions, axis=0, keepdims=True)
mode = mode.reshape([-1])
print(f"정확도: {accuracy_score(ytest, mode)*100}%")

정확도: 86.275%


e. 결론: 이와 같이 작은 모델 여러 개의 예측값 최빈값을 이용하는 방식을 앙상블 학습이라 한다. 결정트리를 이용한 앙상블 학습 모델 중에 하나가 다음에 배울 랜덤 포레스트(random forest) 모델이다.